### Make sure grafwiz and v3io_frames are installed

In [ ]:
!pip install git+https://github.com/v3io/grafwiz --upgrade

In [ ]:
!pip install v3io_frames

### Add environment configurations and import the necessary libraries

In [ ]:
grafana_url = 'http://grafana'
v3io_container = 'bigdata'
stocks_kv_table = 'stocks_kv_table'
stocks_tsdb_table = 'stocks_tsdb_table'

In [ ]:
from grafwiz import *
import os
import v3io_frames as v3f
import pandas as pd

### Generate data for the dataframe

#### Date that will serve as an index for the time series and additional parameters

In [ ]:
import random
import datetime
def generate_date(rows):
    datetimes = [datetime.datetime.today() + (random.random() * datetime.timedelta(days=1)) for i in range(rows)]
    return datetimes

def generate_volume_price(rows):
    volume = [random.randint(0,10000) for i in range(rows)]
    return volume

rows = 345
time = generate_date(rows)
volume = generate_volume_price(rows)
price = generate_volume_price(rows)

#### Create a dataframe out of the generated index and add an index to the dataframe in order to make it accessible for tsdb

In [ ]:
stocks_df = pd.DataFrame(
    {'last_updated': time,
     'volume': volume,
     'price': price
    })
stocks_df['symbol'] = 'GOOGL'
stocks_df = stocks_df.sort_values('last_updated')

stocks_df

In [ ]:
stocks_df = stocks_df.reset_index()
stocks_df = stocks_df.set_index(['last_updated', 'symbol'])


#### Create a client for v3io_frames create table and persist the dataframe in tsdb

In [ ]:
client = v3f.Client('framesd:8081',container='bigdata')
client.create(backend='tsdb', table=stocks_tsdb_table, rate='1/m', if_exists=1)
client.write(backend='tsdb', table=stocks_tsdb_table, dfs=stocks_df)

#### Update the kv table in order of rows arrival to simulate real time changing data

In [ ]:
expr_template = "symbol='{symbol}';price={price};volume={volume};last_updated='{last_updated}'"
# update NoSQL table with stock data
for idx, record in stocks_df.iterrows():
    stock = {'symbol': sym, 'price': record['price'], 'volume': record['volume'], 'last_updated': record['last_updated']}
    expr = expr_template.format(**stock)
    client.execute('kv', stocks_kv_table, 'update', args={'key': sym, 'expression': expr})

#### Create a Datasource for grafana table and check that kv table exists and accessible

In [ ]:
# Create datasources
DataSource(name='Iguazio').deploy(grafana_url, use_auth=True)

# Verify the KV table can be shown
client = v3f.Client('framesd:8081', container=v3io_container)
client.execute(backend='kv', table=stocks_kv_table, command='infer')

#### Create a Dashboard define its datasource and then create a dashboard template

In [ ]:
# Create grafana dashboard
dash = Dashboard("stocks", start='now-7d', dataSource='Iguazio')

# Add a symbol combo box (template) with data from the stocks table
dash.template(name="SYMBOL", label="Symbol", query="fields=symbol;table=stocks/stocks_kv;backend=kv;container=bigdata")

#### Create a table and 2 charts

In [ ]:
# Create a table and log viewer in one row
tbl = Table('Current Stocks Value', span=12).source(table=stocks_kv_table,fields=['symbol','volume', 'price', 'last_updated'],container=v3io_container)
dash.row([tbl])

# Create 2 charts on the second row
metrics_row = [Graph(metric).series(table=stocks_tsdb_table, fields=[metric], filter='symbol=="$SYMBOL"',container=v3io_container) for metric in ['price','volume']]
metrics_row.append(Graph('price').series(table=stocks_tsdb_table, fields=['price'], filter='symbol=="$SYMBOL"', container=v3io_container))
dash.row(metrics_row)




In [ ]:
# Deploy to Grafana
dash.deploy(grafana_url)